In [ ]:
# CRIA O AMBIENTE VIRTUAL

conda create -n PEFT_LoRA python=3.11
conda activate PEFT_LoRA

In [ ]:
# INSTALA AS DEPENDENCIAS

pip install peft trl

pip install ipykernel
python -m ipykernel install --user --name=PEFT_LoRA --display-name="PEFT_LoRA"

In [ ]:
import tqdm as notebook_tqdm
import json
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

In [ ]:
# CONFIGURA OS PARÂMETROS E CARREGA O DATASET

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
DATASET_FILE = "dataset_instrucoes_juridico.jsonl"
OUTPUT_DIR = "./modelo_juridico_adaptado_LoRA"

# Carrega o dataset a partir do arquivo JSONL
train_dataset = load_dataset('text', data_files={'train': DATASET_FILE})
print("Dataset carregado com sucesso:")
print(raw_datasets["train"])

In [ ]:
# CARREGAR MODELO E TOKENIZADOR

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Define o pad_token se não existir
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Ajuste para o modelo Qwen: o pad_token_id deve ser o eos_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Carrega o Modelo em bfloat16 para economizar memória
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,  # Carrega em 16-bit (essencial para GPUs de consumidor)
    device_map="auto",           # Mapeia o modelo para a GPU automaticamente
    trust_remote_code=True
)

print("Modelo e Tokenizador carregados com sucesso em 16-bit (bfloat16).")

In [ ]:
# CONFIGURAÇÃO DO LoRA

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# Imprimir um resumo do modelo para confirmar a pequena porcentagem de parâmetros que serão treinados.
model.print_trainable_parameters()

In [ ]:
# CONFIGURAR E EXECUTAR O TREINAMENTO COM SFTTrainer

# Argumentos de Treinamento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,      # Um batch size pequeno é crucial sem quantização
    gradient_accumulation_steps=4,      # Acumula gradientes para simular um batch maior
    learning_rate=2e-4,                 # Taxa de aprendizado comum para PEFT
    num_train_epochs=5,
    logging_steps=10,
    bf16=True,                          # Ativa o uso de bfloat16 no treinamento (otimização de memória e velocidade)
    save_strategy="epoch",
    overwrite_output_dir=True,
)

# Instanciar o SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_args,
    tokenizer=tokenizer,
)

# Inicia o treinamento
trainer.train()

# Salva o modelo treinado (apenas os adaptadores)
print(f"Treinamento concluído. Salvando os adaptadores em: {OUTPUT_DIR}")
trainer.save_model()

In [ ]:
# VERIFICAÇÃO E DEMONSTRAÇÃO


# --- Prompt de Teste ---
prompt = "A palavra de um colaborador é suficiente para condenar alguém?"
formatted_prompt = f"### Instrução:\n{prompt}\n\n### Resposta:\n"


# --- Carregar o modelo base original para comparação ---
# Reutilizaremos o modelo já carregado na memória, se possível.
base_model = model.get_base_model() 

# --- Carregar o modelo com os adaptadores PEFT ---
# O `peft_model` é o modelo que já está pronto após o trainer.save_model()
# Para garantir, recarregamos do disco.
peft_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# --- Gerar Respostas ---
device = "cuda:0"
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)


print("--- [TESTE 1] Resposta do Modelo Original (sem fine-tuning) ---")
# Usamos o `base_model` para gerar a resposta original
with torch.no_grad():
    outputs = base_model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

print("\n" + "="*50 + "\n")

print("--- [TESTE 2] Resposta do Modelo com Adapters (pós fine-tuning) ---")
# Usamos o `peft_model` para gerar a resposta especializada
with torch.no_grad():
    outputs = peft_model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))